# Projet maison n° 4

In [9]:
# imports
import requests
from bs4 import BeautifulSoup

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [38]:
import re

# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):  
    r = requests.get(url)
    html = r.content
    soup = BeautifulSoup(html)
    
    prices = {}
    offers = soup.find_all('div', {'class': "pricing-table"})
    
    for elem in offers:
        #print(elem)
        offer_elem = {}
        offer_elem["price"] = elem.find_all('span',{'class' : 'pricing-table-price'})[0].text[-14:-11].replace(" ","")
        offer_elem["storage"] = elem.find_all('li')[3].text.split(" ")[0]
        offer_elem["databases"] = int(elem.find_all('li')[4].text.split(" ")[0])
        prices[elem.find("h2").text] = offer_elem
    
    return prices

print(get_prices_from_url(URL_PAGE2))
print(get_prices_from_url(URL_PAGE3))

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}, 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}


**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [59]:
# partie B-1
url = "https://www.beerwulf.com/fr-fr/p/bieres/la-chouffe-blonde-dardenne-2l-keg"

def extract_beer_infos(url):   
    r = requests.get(url)
    html = r.content
    soup = BeautifulSoup(html)
    
    beer_infos = {}
    
    beer_infos["nom"] = soup.find_all('div', {'class': "product-detail-info-title"})[0].find("h1").text
    beer_infos["Style"] = soup.find_all('a', {'href' : "/fr-fr/c/biereblonde"})[0].text
    beer_infos["Contenu"] = int(soup.find_all('dd', {'class' : "js-beer-volume"})[0].text.split(" ")[0])*100
    beer_infos["Degré d’alcool"] = float(soup.find_all('dd',{'class' : "small-6 medium-9 columns"})[1].text[:-1].replace(",","."))
    beer_infos["Origine"] = soup.find_all('dd', {'class' : "js-beer-country"})[0].text[2:].replace(" ","")
    beer_infos["Brasseur"] = soup.find_all('a', {'href' : "/fr-fr/p/brasseurs/brasserie-dachouffe"})[0].text

    return beer_infos

print(extract_beer_infos(url))

{'nom': "La Chouffe Blonde D'ardenne - Fût 2L The SUB", 'Style': 'Bière Blonde', 'Contenu': 200, 'Degré d’alcool': 8.0, 'Origine': 'Belgique', 'Brasseur': "Brasserie D'Achouffe"}


In [83]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

def extract_beer_list_infos(url):
    response = requests.get(url)
    data = response.json()
    
    beers = []
    for elem in data["items"]:
        elem_list = {
            "Nom" : elem["title"],
            "Style" : elem["style"],
            "Contenu" : int(elem["volume"]),
            "Degré d’alcool" : float(elem["alcoholPercentage"]),
            'Origine':'France',
            "Brasseur" : "Brasserie de " + elem["title"]
        }
        beers.append(elem_list)

    return beers
{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'Nom': 'La Cristal IPA du Mont Blanc',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Brasserie de La Cristal IPA du Mont Blanc'},
 {'Nom': 'Ninkasi Pale Ale',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'Brasserie de Ninkasi Pale Ale'},
 {'Nom': 'Desperados',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Brasserie de Desperados'},
 {'Nom': 'Pietra',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'Brasserie de Pietra'}]

In [84]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [85]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.538s

OK
